In [ ]:
def feature_selection_plot(inp_df,inp_feature_count,inp_model_name):
    scaler=MinMaxScaler()
    inp_df.columns=['Features','Importance']
    importance_scaled=pd.DataFrame(scaler.fit_transform(inp_df[['Importance']].values),index=inp_df.index, columns=['Importance'])
    inp_df['Importance']=importance_scaled['Importance']
    inp_df.sort_values(by='Importance', inplace=True, ascending=True)
    df_plot=inp_df.tail(inp_feature_count).reset_index(drop=True)
    fig=go.Figure()
    fig.add_trace(go.Bar(x=df_plot['Importance'],
                        y=df_plot['Features']
                        orientation='h'))
    fig.update_traces(marker_color='steelblue')
    fig.update_traces(title='Importance')
    fig.update_traces(title='Features')
    fig.update_traces(title='Feature selection using {}'.format(inp_model_name))
    fig.show(config={'displaylogo':False})
    return inp_df

In [ ]:
def cross_val_plot(cross_val_scores,model_name,metric_name):
    abs_error=[abs(ele) for ele in cross_val_scores]
    folds=["Fold {}".format(ele) for ele in range(1,len(cross_val_scores)+1)]
    fig=go.Figure([go.Bar(x=folds,y=abs_error,marker_color='steelblue')])
    title='{} fold cross-validation for {}'.format(len(cross_val_scores),model_name)
    fig.update_layout(title=title,
                     xaxis_title='Folds'
                     yaxis_title=metric_name, width=900,height=500)
    fig.show(config={'displaylogo':False})

In [ ]:
def cross_valid_result(classifier,x_data_t,y_data_t,cv_k,params):
    skf=StratifiedKFold(n_splits=cv_k,shuffle=True,random_state=369)
    grid_search=GridSearchCV(classifier,param_grid=params,
                            cv=skf.split(x_data_t,y_data_t),
                            scoring='f1_macro',n_jobs=-1,verbose=3)
    model_opt=grid_searchd_search.fit(x_data_t,y_data_t)
    
    cv_res_df=pd.DataFrame(data=model_opt.cv_results_)
    cv_res_df.drop(['mean_fit_time','std_fit_time','mean_score_time','std_test_score','params'],axis=1,inplace=True)
    cv_res_df_col=list(cv_res_df.columns)
    cv_res_df_col.insert(0,cv_res_df_col.pop())
    cv_res_df=cv_res_df.reindex(columns=cv_res_df_col).sort_values(by='rank_test_score')
    
    opt_params=model_opt.best_params_
    classifier=classifier.set_params(**opt_params)
    final_model=classifier.fit(x_data_t,y_data_t)
    y_pred_t=final_model.predict(x_data_t)
    cv_scores=cross_valid_score(classifier,x_data_t,y_data_t,cv=cv_k,scoring='f1_macro')
    clear_output()
    
    display(Markdown('__Cross validation for TRAIN dataset__'))
    display_data(cv_res_df.round(4))
    
    display(Markdown('__Best parameters__'))
    for k,v in opt_params.items():
        display(Markdown('__{}__ : {}'.format(k,v)))
        
    return final_model,cv_scores,y_pred_t,opt_params
    

In [ ]:
def calculate_cm(cm):
    FP=cm.sum(axis=0)-np.diag(cm)
    FN=cm.sum(axis=1)-np.diag(cm)
    TP=np.diag(cm)
    TN=cm.sum()-(FP+FN+TP)
    
    FP=FP.astype(float)
    FN=FN.astype(float)
    TP=TP.astype(float)
    TN=TN.astype(float)
    
    return FP,FN,TP,TN